In [2]:
## Importing Modules 

from bs4 import BeautifulSoup
import pandas as pd
import requests

In [3]:
## getting the appropriate website parsed into hmtl

url = 'https://www.procyclingstats.com/race/tour-de-france/2023/stage-21'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')


In [4]:

headers = []
data = []
rows = soup.find_all('tr')
header_row = rows[0]
for header in header_row.find_all('th'):
    headers.append(header) 
    data.append([])
for data_row in rows[1:]:
    row = data_row.find_all('td')
    for i in range(len(row)):
        #print(row[i].text.strip())
        if len(row) != len(data):
            continue
        else:
            data[i].append(row[i].text.strip())
data_dict = {}
for i in range(len(headers)):
    data_dict[headers[i]] = data[i]
df = pd.DataFrame(data_dict)
df.head()


,[Rnk],[GC],[Timelag],[BIB],[H2H],[Specialty],[Rider],[Age],[Team],[UCI],[Pnt],[],[Time]
0,1,139,+5:33:51,76,,Classic,MEEUS Jordi BORA - hansgrohe,25,BORA - hansgrohe,210,100,,2:56:13
1,2,97,+4:32:46,106,,Sprint,PHILIPSEN Jasper Alpecin - Deceuninck,25,Alpecin - Deceuninck,150,70,,",,0:00"
2,3,137,+5:27:21,164,,Sprint,GROENEWEGEN Dylan Team Jayco AlUla,30,Team Jayco AlUla,110,50,,",,0:00"
3,4,105,+4:43:50,86,,Classic,PEDERSEN Mads Lidl - Trek,27,Lidl - Trek,90,40,,",,0:00"
4,5,149,+5:57:44,192,,Sprint,BOL Cees Astana Qazaqstan Team,27,Astana Qazaqstan Team,70,32,,",,0:00"
